На основании представленных ссылок на спецификации различных моделей тракторов сделайте нейро-консультанта для ответов на вопросы покупателя, используя динамический выбор из разных баз знаний.

Ссылки для создания БЗ:

https://assets.cnhindustrial.com/nhag/apac/ru-ru/assets/pdf/agriculture-tractors/IR1106NCIS.pdf

http://agromester.md/pdf/9r.pdf

Проверьте работу нейро-консультанта на самостоятельно сформулированных вопросах


In [1]:
!pip install -q langchain_openai==0.1.22 faiss-cpu==1.8.0 openai==1.55.3 tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12 langchain-text-splitters==0.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00


In [2]:
# импортируем необходимые библиотеки
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os
import re
import requests
import openai
import tiktoken
import matplotlib.pyplot as plt
from textwrap import fill
from langchain_openai import ChatOpenAI                   # класс ChatOpenAI
from langchain.prompts import PromptTemplate              # шаблона запроса
from langchain.chains import LLMChain                     # цепочка LLM
from langchain.chains.router import MultiRetrievalQAChain # Импорт многократного поиска

import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)

In [3]:
from google.colab import userdata

# Использование ключа API от ProxyAPI
key = userdata.get('PROXY_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [4]:
# функция принимает URL PDF-файла и путь, по которому файл будет сохранен в файловой системе Colab
# она скачивает файл и сохраняет его, если HTTP-запрос успешен
def download_pdf_from_url(url, save_path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Failed to download PDF. HTTP Status Code: {response.status_code}")

In [5]:
!pip install PyMuPDF==1.25.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.8 MB/s eta 0:00:00


In [6]:
# импорт модулей
import fitz # работа с документами в формате PDF

# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_vectorstore(pdf_names):
    pdf_folder_path = "/content/"

    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=len)

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder_path, pdf_file)

        # Извлечение текста из PDF
        try:
            with fitz.open(pdf_path) as pdf:
                text = ""
                for page in pdf:
                    text += page.get_text()
        except Exception as e:
            print(f"Ошибка при обработке файла {pdf_file}: {e}")
            continue

        # Разделение текста на части
        documents = text_splitter.split_text(text)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(all_documents, embeddings).as_retriever()

    return all_documents, vectorstore

In [7]:
url = "https://assets.cnhindustrial.com/nhag/apac/ru-ru/assets/pdf/agriculture-tractors/IR1106NCIS.pdf"
save_path = "/content/NEW_HOLLAND_T6OOO.pdf"
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/NEW_HOLLAND_T6OOO.pdf


In [8]:
pdf_names = ['NEW_HOLLAND_T6OOO.pdf']
text_db_holland, db_holland = create_faiss_vectorstore(pdf_names)

In [9]:
text_db_holland

['ТРАКТОРЫ NEW HOLLAND T6OOO \nС ТРАНСМИССИЕЙ RANGE COMMAND™ И POWER COMMAND™ \nT6O5O   T6O7O   T6O8O   T6O9O\n2 3\nЛЕГКИЙ И МАНЕВРЕННЫЙ. МОЩНЫЙ И ЭФФЕКТИВНЫЙ\nПО-НАСТОЯЩЕМУ УНИВЕРСАЛЬНЫЙ \nИщите трактор с лучшей на рынке энерговооруженностью? Трактор Т6000 отличает малый вес и исключительный \nмощностный баланс. Стоит принять во внимание непревзойденную маневренность, замечательный круговой обзор \nиз кабины и простоту управления. Нужна многоцелевая машина? Безукоризненная работа ВОМа и широкий диапазон \nходовых передач - все, что необходимо для достижения быстрого и безупречного результата. А теперь сопоставьте \nданные возможности с простотой, скоростью и безопасностью работы трактора в качестве транспорта для перевозки',
 'сельскохозяйственных грузов. Отвечает ли полученный результат вашим представлениям о настоящей многоцелевой машине?\nМОЩЬ И ПРОИЗВОДИТЕЛЬНОСТЬ\nСистема увеличения мощности Power Boost, предусмотренная на всех моделях T6000 с трансмиссией Range Command и \nPower 

In [11]:
url = 'https://agromester.md/pdf/9r.pdf'
save_path = '/content/9R_RT.pdf'
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/9R_RT.pdf


In [12]:
pdf_names = ['9R_RT.pdf']
text_db_9rrt, db_9rrt = create_faiss_vectorstore(pdf_names)

In [13]:
# посмотрим содержимое базы знаний
text_db_9rrt

['Тракторы серии 9R/RT\nОт 302 до 412 кВт (от 410 до 560 л.с.) 97/68EC \n2\n|\nОсвободите зверя\nПредставьте трактор мощностью до 560 л.с. Высокотехнологичный инновационный трактор, \n\xadготовый выполнить тяжелейшие сельскохозяйственные операции с легкостью.\nПознакомьтесь с новым трактором John\xa0Deere серии 9R, который способен ошеломить вас своей \nмощностью.\nПредлагаются четыре колесные и три гусеничные модели – 9410R, 9460R/9460RT, 9510R/9510RT \nили 9560R/9560RT – с двигателями \xadноминальной мощности от 410 л.с. до 560 л.с. и максимальной \nмощности от 451 до to 616 л.с.\nНо эти тракторы отличаются не только мощностью, но и дополнительными характеристиками. \n\xadТакими как комфортабельная, просторная кабина CommandView II и современные интегрирован\xad',
 '\xadТакими как комфортабельная, просторная кабина CommandView II и современные интегрирован\xad\nные функции систем точного земледелия AMS. Эти тракторы являются разумной инвестицией при \nпроведении широкомасштабных сель

In [14]:
retriever_infos = [
    {
        "name": "NEW HOLLAND T6OOO",
        "description": "Suitable for answering questions about NEW HOLLAND T6OOO cars",  # описываем, для каких вопросов применять данную БЗ
        "retriever": db_holland
    },
    {
        "name": "9R RT",
        "description": "Suitable for answering questions about 9R RT cars", # описываем, для каких вопросов применять данную БЗ
        "retriever": db_9rrt
    }
]

In [15]:
system = """
  Очень подробно и детально ответь на вопрос пользователя,
  опираясь точно на документ с информацией для ответа клиенту.
  Не придумывай ничего от себя. Не ссылайся на сами отрывки документа
  с информацией для ответа, клиент о них ничего не должен знать.
"""

In [16]:
# определяем модель
llm = ChatOpenAI(temperature=0)

# шаблон для запроса
prompt = PromptTemplate(input_variables=["query"], template="Question: {query}")

# цепочка, которую будем использовать по умолчанию
default_chain_llm = LLMChain(llm=llm, prompt=prompt)

# создание MultiRetrievalQAChain с указанием default_chain_llm
chain = MultiRetrievalQAChain.from_retrievers(
  llm=llm,
  retriever_infos=retriever_infos,
  default_chain_llm=default_chain_llm,
  verbose=True
)

/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [17]:
print(chain.invoke("Какая максимальная мощньсть 9R RT?")["result"])



> Entering new MultiRetrievalQAChain chain...
9R RT: {'query': 'Какая максимальная мощность 9R RT?'}
> Finished chain.
Максимальная мощность тракторов серии 9R/RT составляет от 451 до 616 л.с. (97/68 EC).


In [18]:
print(chain.invoke("Какая максимальная скорость T6000?")["result"])



> Entering new MultiRetrievalQAChain chain...
NEW HOLLAND T6OOO: {'query': 'Какая максимальная скорость T6000?'}
> Finished chain.
Максимальная скорость движения для New Holland T6000 может достигать 40 км/ч при уменьшенном числе оборотов двигателя.
